<a href="https://colab.research.google.com/github/Krishanu-Saha/data-science/blob/main/Regression_analysis_on_Nairobi_Transport_demand.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project Name**    - 



##### **Project Type**    - Regression
##### **Contribution**    - Individual


# **Project Summary -**

Write the summary here within 500-600 words.

# **GitHub Link -**

Provide your GitHub Link here.

# **Problem Statement**


**Rossmann operates over 3,000 drug stores in 7 European countries. Currently, Rossmann store managers are tasked with predicting their daily sales for up to six weeks in advance. Store sales are influenced by many factors, including promotions, competition, school and state holidays, seasonality, and locality. With thousands of individual managers predicting sales based on their unique circumstances, the accuracy of results can be quite varied. We are provided with historical sales data for 1,115 Rossmann stores. The task is to forecast the "Sales" column for the test set.**

# **General Guidelines** : -  

1.   Well-structured, formatted, and commented code is required. 
2.   Exception Handling, Production Grade Code & Deployment Ready Code will be a plus. Those students will be awarded some additional credits. 
     
     The additional credits will have advantages over other students during Star Student selection.
       
             [ Note: - Deployment Ready Code is defined as, the whole .ipynb notebook should be executable in one go
                       without a single error logged. ]

3.   Each and every logic should have proper comments.
4. You may add as many number of charts you want. Make Sure for each and every chart the following format should be answered.
        

```
# Chart visualization code
```
            

*   Why did you pick the specific chart?
*   What is/are the insight(s) found from the chart?
* Will the gained insights help creating a positive business impact? 
Are there any insights that lead to negative growth? Justify with specific reason.

5. You have to create at least 15 logical & meaningful charts having important insights.


[ Hints : - Do the Vizualization in  a structured way while following "UBM" Rule. 

U - Univariate Analysis,

B - Bivariate Analysis (Numerical - Categorical, Numerical - Numerical, Categorical - Categorical)

M - Multivariate Analysis
 ]





6. You may add more ml algorithms for model creation. Make sure for each and every algorithm, the following format should be answered.


*   Explain the ML Model used and it's performance using Evaluation metric Score Chart.


*   Cross- Validation & Hyperparameter Tuning

*   Have you seen any improvement? Note down the improvement with updates Evaluation metric Score Chart.

*   Explain each evaluation metric's indication towards business and the business impact pf the ML model used.




















# ***Let's Begin !***

## ***1. Know Your Data***

### Import Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Import Libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from datetime import datetime
import statsmodels.api as sm


### Dataset Loading

In [ ]:
# Load Dataset
sales_df = pd.read_csv('/content/drive/MyDrive/Almabetter /project/REGRESSION/Rossmann Stores Data.csv')
stores_df = pd.read_csv('/content/drive/MyDrive/Almabetter /project/REGRESSION/store.csv')

### Dataset First View

In [ ]:
# Dataset First Look
sales_df.head()

In [ ]:
stores_df.head()

### Dataset Rows & Columns count

In [ ]:
# Dataset Rows & Columns count
sales_df.shape

In [ ]:
stores_df.shape

Our dataset contains 51645 rows and 10 columns 

### Dataset Information

In [ ]:
stores_df.info()

We have zero null values in every column.
We need to convert seat_number , travel_date and travel_time from object to their respective data types.

#### Duplicate Values

In [ ]:
len(stores_df[stores_df.duplicated()])

We have zero duplicate rows.Well thats a good sign!

#### Missing Values/Null Values

In [ ]:
stores_df.isnull().sum()

In [ ]:
# filling competition distance with the median value
stores_df['CompetitionDistance'].fillna(stores_df['CompetitionDistance'].median(), inplace = True)
     

In [ ]:
# filling competition open since month and year with the most occuring values of the columns i.e modes of those columns
stores_df['CompetitionOpenSinceMonth'].fillna(stores_df['CompetitionOpenSinceMonth'].mode()[0], inplace = True)
stores_df['CompetitionOpenSinceYear'].fillna(stores_df['CompetitionOpenSinceYear'].mode()[0], inplace = True)

In [ ]:
# imputing the nan values of promo2 related columns with 0
stores_df['Promo2SinceWeek'].fillna(value=0,inplace=True)
stores_df['Promo2SinceYear'].fillna(value=0,inplace=True)
stores_df['PromoInterval'].fillna(value=0,inplace=True)
     

In [ ]:
#merge the datasets on stores data
df = sales_df.merge(right=stores_df, on="Store", how="left")
     

In [ ]:
#first five rows of the merged dataset
df.head()
     

In [ ]:

#shape of the dataframe
df.shape

In [ ]:
#datatypes
df.info()

We need to change certain column datatypes .date ,Stateholiday

In [ ]:
df.nunique()

In [ ]:
#change into int type
df['StateHoliday'] = df['StateHoliday'].replace({'0':0,'a':1,'b':1,'c':1})

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:

#creating features from the date
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['WeekOfYear'] = df['Date'].dt.weekofyear
df['DayOfYear'] = df['Date'].dt.dayofyear
years = df['Year'].unique()
years

### What did you know about your dataset?

We are given with the dataset which have 1017209 rows and 9 columns.Our target variable is 'Sales' column. we have zero duplicated or null rows.

## ***2. Understanding Your Variables***

In [ ]:
# Dataset Columns
columns = list(df.columns)

In [ ]:
# Dataset Describe
df.describe()

In [ ]:
df.info()

### Variables Description 

Id :Unique entry id

Store :store_id

Sales:Sales made for the day

Customers: Footfall for the day

Open: Open or closed

StateHoliday: State Holiday or not

SchoolHoliday: School Holiday or not

Store: TypeType of stores

Assortment :Type of assortment

Competition :DistanceDistance from the nearest competition

PromoStore :running promotion or not

Promo2 :Store running consecutive promotion or not





### Check Unique Values for each variable.

In [ ]:
# Check Unique Values for each variable.
df.nunique()

## 3. ***Data Wrangling***

### Data Wrangling Code

In [ ]:
# Drop all rows where "Open" is equal to zero
data = df[df['Open'] != 0]

In [ ]:
# Group the data by the Promo column and calculate the total number of customers for each Promo category
customers_by_promo = data.groupby(['Promo'])['Customers'].sum().reset_index()

# Create a bar plot with the Promo categories on the x-axis and the total number of customers on the y-axis
plt.bar(customers_by_promo['Promo'], customers_by_promo['Customers'])
plt.title('Total Customers by Promo')
plt.xlabel('Promo')
plt.ylabel('Customers')
plt.show()

In [ ]:
customers_by_promo

In [ ]:
# Group the data by the Promo column and calculate the total number of customers for each Promo category
customers_by_promo = data.groupby(['StateHoliday'])['Customers'].sum().reset_index()

# Create a bar plot with the Promo categories on the x-axis and the total number of customers on the y-axis
plt.bar(customers_by_promo['StateHoliday'], customers_by_promo['Customers'])
plt.title('Total Customers by StateHoliday')
plt.xlabel('StateHoliday')
plt.ylabel('Customers')
plt.show()

In [ ]:
# Group the data by the Promo column and calculate the total number of customers for each Promo category
customers_by_promo = data.groupby(['SchoolHoliday'])['Customers'].sum().reset_index()

# Create a bar plot with the Promo categories on the x-axis and the total number of customers on the y-axis
plt.bar(customers_by_promo['SchoolHoliday'], customers_by_promo['Customers'])
plt.title('Total Customers by SchoolHoliday')
plt.xlabel('SchoolHoliday')
plt.ylabel('Customers')
plt.show()

In [ ]:
# Group the data by the Promo column and calculate the total number of customers for each Promo category
customers_by_promo = data.groupby(['StoreType'])['Customers'].sum().reset_index()

# Create a bar plot with the Promo categories on the x-axis and the total number of customers on the y-axis
plt.bar(customers_by_promo['StoreType'], customers_by_promo['Customers'])
plt.title('Total Customers by StoreType')
plt.xlabel('StoreType')
plt.ylabel('Customers')
plt.show()

In [ ]:
# Group the data by the Promo column and calculate the total number of customers for each Promo category
customers_by_promo = data.groupby(['Assortment'])['Customers'].sum().reset_index()

# Create a bar plot with the Promo categories on the x-axis and the total number of customers on the y-axis
plt.bar(customers_by_promo['Assortment'], customers_by_promo['Customers'])
plt.title('Total Customers by Assortment')
plt.xlabel('Assortment')
plt.ylabel('Customers')
plt.show()

In [ ]:
# Define the list of columns to plot
col_count = ['DayOfWeek',  'Promo', 'StateHoliday', 'SchoolHoliday',
             'StoreType', 'Assortment', 'CompetitionOpenSinceMonth',
             'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
             'Promo2SinceYear', 'PromoInterval', 'Year', 'Month',
             'WeekOfYear']

# Create bar plots of the mean sales for each value in each column
for col in col_count:
    plt.figure(figsize=(12,6))
    data.groupby(col)['Sales'].mean().plot(kind='bar')
    plt.title('Mean Sales vs. ' + col)
    plt.xlabel(col)
    plt.ylabel('Mean Sales')
    plt.show()

In [ ]:
# Calculate the counts of each store type
store_counts = data['StoreType'].value_counts()

# Create a pie chart using the store type counts
plt.figure(figsize = (12,8))
plt.pie(store_counts, labels=store_counts.index, autopct='%1.1f%%')

# Set the title of the plot
plt.title('Store Types')

# Display the plot
plt.show()

In [ ]:
#Let's check the relationship between store type, assortment levels and sales
sns.barplot(x=data["StoreType"],y=data['Sales'],hue=df["Assortment"])

In [ ]:
# Convert the Year and Month columns into a single date column
data['Date'] = pd.to_datetime(data['Year'].astype(str) + '-' + data['Month'].astype(str) + '-1')

# Group the data by year and month and compute the total sales for each group
sales_by_year_month = df.groupby(['Year', 'Month'])['Sales'].sum()

# Create a figure and axis for the plot
fig, ax = plt.subplots(figsize=(10, 5))

# Loop through each year and plot the monthly sales as a line on the same axis
for year in sales_by_year_month.index.levels[0]:
    sales_by_month = sales_by_year_month.loc[year]
    ax.plot(sales_by_month.index, sales_by_month.values, label=str(year))

# Add axis labels and a legend to the plot
ax.set_xlabel('Month')
ax.set_ylabel('Total Sales')
ax.set_title('Monthly Sales Over Time by Year')
ax.legend()

# Display the plot
plt.show()

In [ ]:
data.columns


In [ ]:
num_columns = [ 'DayOfWeek', 'Sales', 'Customers',  'CompetitionDistance', 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Year', 'Month', 'WeekOfYear',  'DayOfYear']
        
 # Create a correlation matrix of the numerical columns
correlation_matrix = data[num_columns].corr()

# Create the heatmap
plt.figure(figsize=(10,5))
sns.heatmap(correlation_matrix, annot=True)

# Set the title of the plot
plt.title('Correlation Heatmap')

# Display the plot
plt.show()      
     

### HYPOTHESIS TESTING 

**1) The effect of promotions on sales: We are testing whether there is a significant difference in sales between days when there is a promotion versus days when there is no promotion.**

Null hypothesis: There is no significant difference in sales between stores with and without a promotion.

Alternative hypothesis: Stores with a promotion have significantly higher sales than stores without a promotion.

In [ ]:
df = data.copy()

In [ ]:
import scipy.stats as stats

# Filter the data to include only days with promotions and non-promotions
promo_sales = df[df['Promo']==1]['Sales']
nonpromo_sales = df[df['Promo']==0]['Sales']

# Test for difference in means using two-sample t-test
t, p = stats.ttest_ind(promo_sales, nonpromo_sales, equal_var=False)
print('t-value: {:.2f}, p-value: {:.4f}'.format(t, p))

**The effect of competition on sales: We are testing whether there is a significant correlation between the distance to the nearest competitor and sales.**

Null hypothesis: There is no significant difference in sales between different store types.

Alternative hypothesis: Some store types have significantly higher sales than others.

In [ ]:
from scipy.stats import pearsonr

# Calculate the Pearson correlation coefficient and p-value between sales and competition distance
corr, p = pearsonr(df['Sales'], df['CompetitionDistance'])
print('Correlation coefficient: {:.2f}, p-value: {:.4f}'.format(corr, p))

**The effect of store type on sales: You could test whether there is a significant difference in sales between different types of stores.**

Null hypothesis: There is no significant relationship between competition distance and store sales.

Alternative hypothesis: Stores located closer to competitors have significantly lower sales than stores located farther away.

In [ ]:
from scipy.stats import f_oneway

# Filter the data to include only the three store types
store_a_sales = df[df['StoreType']=='a']['Sales']
store_b_sales = df[df['StoreType']=='b']['Sales']
store_c_sales = df[df['StoreType']=='c']['Sales']

# Test for difference in means using one-way ANOVA
f, p = f_oneway(store_a_sales, store_b_sales, store_c_sales)
print('F-value: {:.2f}, p-value: {:.4f}'.format(f, p))

### What all manipulations have you done and insights you found?

Answer Here.

## ***6. Feature Engineering & Data Pre-processing***

### 1. Handling Missing Values

In [ ]:
# Handling Missing Values & Missing Value Imputation

#### What all missing value imputation techniques have you used and why did you use those techniques?

Answer Here.

### 2. Handling Outliers

In [ ]:
# Handling Outliers & Outlier treatments

##### What all outlier treatment techniques have you used and why did you use those techniques?

Answer Here.

### 3. Categorical Encoding

In [ ]:
# Encode your categorical columns
# Convert categorical variables to numerical using one-hot encoding
df2 = data.copy()



In [ ]:
df2.head()

In [ ]:
df2.columns 

#### What all categorical encoding techniques have you used & why did you use those techniques?

Answer Here.

### 4. Textual Data Preprocessing 
(It's mandatory for textual dataset i.e., NLP, Sentiment Analysis, Text Clustering etc.)

#### 1. Expand Contraction

In [ ]:
# Expand Contraction

#### 2. Lower Casing

In [ ]:
# Lower Casing

#### 3. Removing Punctuations

In [ ]:
# Remove Punctuations

#### 4. Removing URLs & Removing words and digits contain digits.

In [ ]:
# Remove URLs & Remove words and digits contain digits

#### 5. Removing Stopwords & Removing White spaces

In [ ]:
# Remove Stopwords

In [ ]:
# Remove White spaces

#### 6. Rephrase Text

In [ ]:
# Rephrase Text

#### 7. Tokenization

In [ ]:
# Tokenization

#### 8. Text Normalization

In [ ]:
# Normalizing Text (i.e., Stemming, Lemmatization etc.)

##### Which text normalization technique have you used and why?

Answer Here.

#### 9. Part of speech tagging

In [ ]:
# POS Taging

#### 10. Text Vectorization

In [ ]:
# Vectorizing Text

##### Which text vectorization technique have you used and why?

Answer Here.

### 4. Feature Manipulation & Selection

#### 1. Feature Manipulation

In [ ]:
# Convert CompetitionOpenSinceYear and CompetitionOpenSinceMonth to integers
df2['CompetitionOpenSinceYear'] = df2['CompetitionOpenSinceYear'].astype(int)
df2['CompetitionOpenSinceMonth'] = df2['CompetitionOpenSinceMonth'].astype(int)

# Combine CompetitionOpenSinceYear and CompetitionOpenSinceMonth into a single feature
df2['CompetitionOpenSince'] = pd.to_datetime(dict(year=df['CompetitionOpenSinceYear'], month=df['CompetitionOpenSinceMonth'], day=1))

# Drop original columns
df2.drop(['CompetitionOpenSinceYear', 'CompetitionOpenSinceMonth'], axis=1, inplace=True)



This code converts the CompetitionOpenSinceYear and CompetitionOpenSinceMonth columns to integers, then uses pd.to_datetime to combine them into a single column named CompetitionOpenSince as a datetime object. Finally, the original columns are dropped from the DataFrame using the drop method.

In [ ]:
#changing promo2 features into meaningful inputs
#combining promo2 to total months
df2['Promo2Open'] = (df2['Year'] - df2['Promo2SinceYear'])*12 + (df2['WeekOfYear'] - df2['Promo2SinceWeek'])*0.230137

#correcting the neg values
df2['Promo2Open'] = df2['Promo2Open'].apply(lambda x:0 if x < 0 else x)*df2['Promo2']

#creating a feature for promo interval and checking if promo2 was running in the sale month
def promo2running(df):
  month_dict = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', 7:'Jul', 8:'Aug', 9:'Sept', 10:'Oct', 11:'Nov', 12:'Dec'}
  try:
    months = df['PromoInterval'].split(',')
    if df['Month'] and month_dict[df['Month']] in months:
      return 1
    else:
      return 0
  except Exception:
    return 0

#Applying 
df2['Promo2Active'] = df2.apply(promo2running,axis=1)*df2['Promo2']

#Dropping unecessary columns
df2.drop(['Promo2SinceYear','Promo2SinceWeek'],axis=1,inplace=True)

In the code above, we first convert the Promo2SinceWeek and Promo2SinceYear columns to a datetime format using the to_datetime function. We use a lambda function to combine these columns into a single string in ISO year-week format, which is then parsed using the format argument.

We then create the Promo2Active feature by checking whether the store is running a Promo2 campaign, whether the Promo2 start date is less than or equal to the current date, and whether the current month is included in the PromoInterval. The resulting boolean value is then cast to an integer (1 for True, 0 for False).

Finally, we drop the Promo2Since column since it is no longer needed.

To create a new feature called CompetitionDistanceGroup, which groups the CompetitionDistance into several bins, we can use the pandas cut() function.

First, we can create a list of bin edges and labels that define the groupings. Here, let's use the edges [0, 500, 1500, 3000, 5000, np.inf] and the labels ['near', 'medium', 'far', 'very far', 'extreme'].

Then, we can apply the cut() function to the CompetitionDistance column to create a new column called CompetitionDistanceGroup.

In [ ]:
# Define the bin edges and labels
bin_edges = [0, 500, 1500, 3000, 5000, np.inf]
bin_labels = ['near', 'medium', 'far', 'very far', 'extreme']

# Create the CompetitionDistanceGroup column
df2['CompetitionDistanceGroup'] = pd.cut(df2['CompetitionDistance'], bins=bin_edges, labels=bin_labels)

# Show the first 5 rows of the new column
print(df2[['CompetitionDistance', 'CompetitionDistanceGroup']].head())

In [ ]:
#This will create a new column in the dataframe called 'AvgSalesPerCustomer', which will contain the average sales per customer for each store.
df2['AvgSalesPerCustomer'] = df2['Sales'] / df2['Customers']

In [ ]:
df2.head()

#### 2. Feature Selection

In [ ]:
# Calculate the correlation matrix
corr_matrix = df2.corr()

# Select the features with a correlation coefficient greater than 0.5
high_corr_features = corr_matrix.index[abs(corr_matrix['Sales']) > 0.5]

# Print the selected features
print(high_corr_features)

Since in correlation matrix only column customer is highly correlated with the Sales column.We have to select features from different method.

In [ ]:
# Feature selection
selected_features = ['Store', 'DayOfWeek', 'Promo', 'StateHoliday', 'SchoolHoliday', 'CompetitionDistance', 'CompetitionOpenSince', 'Promo2Active', 'AvgSalesPerCustomer']

# Create the feature matrix
X = df2[selected_features].copy()

# Convert StateHoliday to a categorical variable
X['StateHoliday'] = X['StateHoliday'].astype('category')

# Convert DayOfWeek to a categorical variable
X['DayOfWeek'] = X['DayOfWeek'].astype('category')

# Convert Promo2Active to a categorical variable
X['Promo2Active'] = X['Promo2Active'].astype('category')

# Convert CompetitionOpenSince to a datetime variable
X['CompetitionOpenSince'] = pd.to_datetime(X['CompetitionOpenSince'], format='%Y-%m')

# Extract the year and month of CompetitionOpenSince
X['CompetitionOpenSinceYear'] = X['CompetitionOpenSince'].dt.year
X['CompetitionOpenSinceMonth'] = X['CompetitionOpenSince'].dt.month

# Remove the original CompetitionOpenSince column
X = X.drop('CompetitionOpenSince', axis=1)

# One-hot encode categorical variables
X = pd.get_dummies(X)

# Replace missing values with 0
X = X.fillna(0)

# Print the first 5 rows of the feature matrix
print(X.head())

In [ ]:
# Target variable 
Y = df2['Sales'].copy()

In [ ]:
X = sm.add_constant(X)
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


In [ ]:
model_1 = sm.OLS(y_train,x_train).fit()
model_1.summary2()

In [ ]:
df3 = X.copy()

In [ ]:
df3.info()

### **CHECKING MULTICOLLINEARITY**

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

def get_vif_factors(X):
  X_matrix = X.to_numpy()
  vif =  [variance_inflation_factor(X_matrix,i) for i in range(X_matrix.shape[1])]
  vif_factors = pd.DataFrame()
  vif_factors['column'] = X.columns
  vif_factors['VIF'] = vif

  return vif_factors

In [ ]:
df3.columns
features = ['Store', 'DayOfWeek', 'Date', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'Promo2', 'PromoInterval', 'Year', 'Month',
       'WeekOfYear', 'DayOfYear', 'CompetitionOpenSince', 'Promo2Open',
       'Promo2Active', 'CompetitionDistanceGroup', 'AvgSalesPerCustomer']

In [ ]:
vif_factors = get_vif_factors(df3)
vif_factors

In [ ]:
# select the top 10 features based on f_regression test
from sklearn.feature_selection import SelectKBest, f_regression
selector = SelectKBest(f_regression, k=10)
selector.fit(X, y)

In [ ]:
# get the selected feature indices
selected_features = selector.get_support(indices=True)
selected_feature_names = X.columns[selected_features]
selected_feature_names

**validating the selected features using appropriate evaluation metrics and cross-validation techniques to ensure that the model is not overfitting to the training data.**

In [ ]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression

# create a linear regression model
model = LinearRegression()

# select the top 10 features
X_selected = X[selected_feature_names]

# perform 10-fold cross-validation with MAE metric
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
mae_scores = -cross_val_score(model, X_selected, y, cv=kfold, scoring='neg_mean_absolute_error')

# calculate the mean and standard deviation of the MAE scores
mae_mean = mae_scores.mean()
mae_std = mae_scores.std()

print(f'MAE: {mae_mean:.2f} +/- {mae_std:.2f}')

In [ ]:
import statsmodels.api as sm

model = sm.OLS(y,X_selected).fit()
model.summary2()

A standard deviation of 6.83 for a mean absolute error of 2030.51 indicates that the model's performance may be somewhat variable across different test sets. 

##### What all feature selection methods have you used  and why?

SelectKBest is a feature selection method in scikit-learn that selects the top k features based on univariate statistical tests.

The k parameter in SelectKBest specifies the number of top features to select. The specific test used to score the features is specified by the score_func parameter, which takes a function that scores the features based on their relationship with the target variable.

In this case, f_regression is used as the scoring function, which performs an analysis of variance (ANOVA) F-test to determine the strength of the linear relationship between each feature and the target variable. The get_support() method is used to return a Boolean mask indicating which of the features are selected.

The selector.fit(X, y) method fits the feature selector to the data, using the specified input feature matrix X and target variable y.

During the fitting process, the selector will evaluate each feature in X and assign a score based on its relevance to predicting the target variable. The score is computed using the specified metric or test, in this case f_regression.

Once the scores are computed, the selector selects the top k features with the highest scores, where k is the number of features specified in the k parameter.

Finally, the selector.get_support(indices=True) method returns an array of boolean values indicating which features were selected, where True represents a selected feature and False represents a non-selected feature. The indices=True parameter returns the indices of the selected features.

##### Which all features you found important and why?

Answer Here.

### 5. Data Transformation

#### Do you think that your data needs to be transformed? If yes, which transformation have you used. Explain Why?

In [ ]:
# Transform Your data

### 6. Data Scaling

In [ ]:
# Scaling your data

##### Which method have you used to scale you data and why?

### 7. Dimesionality Reduction

##### Do you think that dimensionality reduction is needed? Explain Why?

Answer Here.

In [ ]:
# DImensionality Reduction (If needed)

##### Which dimensionality reduction technique have you used and why? (If dimensionality reduction done on dataset.)

Answer Here.

### 8. Data Splitting

In [ ]:
# Split your data to train and test. Choose Splitting ratio wisely.

##### What data splitting ratio have you used and why? 

Answer Here.

### 9. Handling Imbalanced Dataset

##### Do you think the dataset is imbalanced? Explain Why.

Answer Here.

In [ ]:
# Handling Imbalanced Dataset (If needed)

##### What technique did you use to handle the imbalance dataset and why? (If needed to be balanced)

Answer Here.

## ***7. ML Model Implementation***

### ML Model - 1

In [ ]:
# ML Model - 1 Implementation

# Fit the Algorithm

# Predict on the model

#### 1. Explain the ML Model used and it's performance using Evaluation metric Score Chart.

In [ ]:
# Visualizing evaluation Metric Score chart

#### 2. Cross- Validation & Hyperparameter Tuning

In [ ]:
# ML Model - 1 Implementation with hyperparameter optimization techniques (i.e., GridSearch CV, RandomSearch CV, Bayesian Optimization etc.)

# Fit the Algorithm

# Predict on the model

##### Which hyperparameter optimization technique have you used and why?

Answer Here.

##### Have you seen any improvement? Note down the improvement with updates Evaluation metric Score Chart.

Answer Here.

### ML Model - 2

#### 1. Explain the ML Model used and it's performance using Evaluation metric Score Chart.

In [ ]:
# Visualizing evaluation Metric Score chart

#### 2. Cross- Validation & Hyperparameter Tuning

In [ ]:
# ML Model - 1 Implementation with hyperparameter optimization techniques (i.e., GridSearch CV, RandomSearch CV, Bayesian Optimization etc.)

# Fit the Algorithm

# Predict on the model

##### Which hyperparameter optimization technique have you used and why?

Answer Here.

##### Have you seen any improvement? Note down the improvement with updates Evaluation metric Score Chart.

Answer Here.

#### 3. Explain each evaluation metric's indication towards business and the business impact pf the ML model used.

Answer Here.

### ML Model - 3

In [ ]:
# ML Model - 3 Implementation

# Fit the Algorithm

# Predict on the model

#### 1. Explain the ML Model used and it's performance using Evaluation metric Score Chart.

In [ ]:
# Visualizing evaluation Metric Score chart

#### 2. Cross- Validation & Hyperparameter Tuning

In [ ]:
# ML Model - 3 Implementation with hyperparameter optimization techniques (i.e., GridSearch CV, RandomSearch CV, Bayesian Optimization etc.)

# Fit the Algorithm

# Predict on the model

##### Which hyperparameter optimization technique have you used and why?

Answer Here.

##### Have you seen any improvement? Note down the improvement with updates Evaluation metric Score Chart.

Answer Here.

### 1. Which Evaluation metrics did you consider for a positive business impact and why?

Answer Here.

### 2. Which ML model did you choose from the above created models as your final prediction model and why?

Answer Here.

### 3. Explain the model which you have used and the feature importance using any model explainability tool?

Answer Here.

## ***8.*** ***Future Work (Optional)***

### 1. Save the best performing ml model in a pickle file or joblib file format for deployment process.


In [ ]:
# Save the File

### 2. Again Load the saved model file and try to predict unseen data for a sanity check.


In [ ]:
# Load the File and predict unseen data.

### ***Congrats! Your model is successfully created and ready for deployment on a live server for a real user interaction !!!***

# **Conclusion**

Write the conclusion here.

### ***Hurrah! You have successfully completed your Machine Learning Capstone Project !!!***